# Break substitution cipher
This notebook demonstrates how to recover keys from messages encrypted under the substitution cipher.

In [ ]:
import sys
sys.path.append('src')

from src.eval_utils import *
from src.data_utils import *
from src.functional import *
import src.alphabet as alph
import src.cipher as Cipher
import src.cracker as Cracker
import seaborn as sns
import random, copy

In [ ]:
# Set parameters
data_id = 'bible-test'
alphabet = alph.basic_lower()
cipher = Cipher.Substitution(alphabet)

In [ ]:
# Samples messages
n_samples = 5
msgs = load_msgs(data_id, alphabet, 0, 0, lower=True)
samples = random.sample(msgs, n_samples)
order = get_char_order(msgs, alphabet)

In [ ]:
# Encrypt samples
subst_cipher = Cipher.Substitution(alphabet)
encs, correct_keys = subst_cipher.encrypt_all(samples, order=order, key_rot=1, key_gen_strategy='random')

In [ ]:
# Initialize cracker with partial decryption classifiers
partial_lengths = list(range(2, 27, 2))

models_dict = {}
for l in partial_lengths:
    model_dict = {l: load_model(f'models/bible_partial{l}_prop{1}_lowdist_shuffle6.cnn', len(alphabet))}

cracker = Cracker.Substitution(alphabet, model_dict)

In [ ]:
predicted_keys = []
n_top=10
for enc in encs:
    predicted_keys.append(cracker.key_search(enc, order, n_top=n_top))

In [ ]:
# Show results
for i, (key, pred) in enumerate(zip(correct_keys, predicted_keys)):
    print(f'### Encryption {i+1}###')
    print(f'Correct key: \t{key}')
    print(f'Top {n_top} predicted keys: \n{[pred.Key]}')
    print(f'Correct key in top {n_top}: {key in pred.Key.values}')
    print()